## Homework #4

## Guy Cutting

### Original model:

In [1]:
import pandas as pd
import numpy as np
np.random.seed(1337) 
import matplotlib.pyplot as plt

from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop

img_rows, img_cols = 28, 28

batch_size = 100 
nb_classes = 10 
nb_epoch = 30

# Read the train and test datasets
train = pd.read_csv("./small_train.csv",header = None).values
test  = pd.read_csv("./small_test.csv",header = None).values

# Check Keras backend
if(K.image_dim_ordering()=="th"): # for Theano
    X_train = train[:, 1:].reshape(train.shape[0], 1, img_rows, img_cols)
    X_test = test[:, 1:].reshape(test.shape[0], 1, img_rows, img_cols)
    in_shape = (1, img_rows, img_cols)
else:  # for TensorFlow
    X_train = train[:, 1:].reshape(train.shape[0], img_rows, img_cols, 1)
    X_test = test[:, 1:].reshape(test.shape[0], img_rows, img_cols, 1)
    in_shape = (img_rows, img_cols, 1)

# First data is label (already removed from X_train)
y_train = train[:, 0] 

# Make the value floats in [0;1] instead of int in [0;255]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices (ie one-hot vectors)
Y_train = np_utils.to_categorical(y_train, nb_classes)

# Display the shapes to check if everything's ok
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_test shape:', X_test.shape)

model = Sequential()

# Add padding to take 28x28 to 32x32
model.add(ZeroPadding2D((2,2),input_shape=in_shape))

# Roughly equivalent to C1
model.add(Convolution2D(6, (5, 5), activation = 'relu',  kernel_initializer='he_normal'))

# Roughly equivalent to S2
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Activation("sigmoid"))

# model.add(Convolution2D(6, (2, 2), stride = (2,2),  activation = 'sigmoid', kernel_initializer='he_normal'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# Roughly equivalent to C3
model.add(Convolution2D(16, (5, 5), activation = 'relu', kernel_initializer='he_normal'))

# Roughly equivalent to S4
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Activation("sigmoid"))

model.add(Dropout(0.2))

# model.add(Convolution2D(16, (2, 2), stride = (2,2),  activation = 'sigmoid', kernel_initializer='he_normal'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# Roughly equivalent to C5
model.add(Convolution2D(120, (5, 5), activation = 'relu', kernel_initializer='he_normal'))
model.add(Flatten())

# Roughly equivalent to F6
model.add(Dense(84, activation = 'tanh', kernel_initializer='he_normal'))


# Output Layer
model.add(Dense(nb_classes, activation = 'softmax', kernel_initializer='he_normal')) #Last layer with one output per class

# Use RMSprop for training weights
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=["accuracy"])

# Alternative training approach using stochastic gradient descent (very very slow)
# sgd = SGD(lr=0.4,momentum=0.1)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])

# Let's Learn!!
model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1)

# Use the test data to see how we do
yPred = model.predict_classes(X_test)


Using TensorFlow backend.


X_train shape: (5000, 28, 28, 1)
Y_train shape: (5000, 10)
X_test shape: (1000, 28, 28, 1)
Epoch 1/30
5000/5000 [==============================] - 4s - loss: 2.3580 - acc: 0.0942      

In [ ]:
# Line up our outputs on the test set with the labels from the test set and calculate a confusion matrix

targets = test[:,0]

cm = np.array([[0] * 10] * 10)
for i in range(len(targets)):
 cm[yPred[i],targets[i]] += 1

print(cm)

In [ ]:
#  Lets look at the ones we got WRONG in thte test set

test_wrong = [im for im in zip(X_test,yPred,test[:,0]) if im[1] != im[2]]

plt.figure(figsize=(10, 10))
for ind, val in enumerate(test_wrong[:100]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(10, 10, ind + 1)
    im = 1 - val[0].reshape((28,28))
    plt.axis("off")
    plt.text(0, 0, val[2], fontsize=14, color='blue')
    plt.text(8, 0, val[1], fontsize=14, color='red')
    plt.imshow(im, cmap='gray')

plt.show()

In [ ]:
#  Lets look at some of the ones we got RIGHT in thte test set

test_wrong = [im for im in zip(X_test,yPred,test[:,0]) if im[1] == im[2]]

plt.figure(figsize=(10, 10))
for ind, val in enumerate(test_wrong[:100]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(10, 10, ind + 1)
    im = 1 - val[0].reshape((28,28))
    plt.axis("off")
    plt.text(0, 0, val[2], fontsize=14, color='blue')
    plt.text(8, 0, val[1], fontsize=14, color='red')
    plt.imshow(im, cmap='gray')

plt.show()

### Initial impressions and notes

The performance on this implementation seems MUCH worse than we would expect just from minor differences between the original LeNet-5 implementation and this one. The accuracy of the LeNet-5 by Lecun et al over 99\% as given here:

http://yann.lecun.com/exdb/mnist/index.html

There are a couple of versions of LeNet-5 listed above but all have less than 1% error rate. One of these models was trained with a dataset containing intentionally distorted images, and still achieved an error rate of ~.8%. See p.13 of the Lecun paper for details on this.

The performance of this implementation is much worse than even for basic toy models. This TensorFlow tutorial:
https://www.tensorflow.org/get_started/mnist/beginners

uses about the most basic possible model and still achieves ~92% accuracy on the MNIST data with a MUCH shorter training time. 

There are several examples of MNIST NNs here:
https://github.com/fchollet/keras/tree/master/examples

All of them achieve accuracy rates in the high 90's, and at least a couple of them trained in a fraction of the time as the LeNet implementation given here. So there is clearly something in this implementation that is significantly reducing performance from what we would expect from a multilayer CNN.

And this page:
https://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/

implements LeNet in Keras with results much closer to LeCun's (~99% accuarcy).

One thing I notice is that the above model is using a small training set (5000). Most other implementations I have seen use much larger training sets (30,000 or 60,000). This is something to try to attempt to improve performance.

So the first thing I want to try is running the above model with a larger training set. 

In [ ]:
img_rows, img_cols = 28, 28

batch_size = 100 
nb_classes = 10 
nb_epoch = 4

# Read the train and test datasets
train = pd.read_csv("./mnist_train.csv",header = None).values
test  = pd.read_csv("./mnist_test.csv",header = None).values

# Check Keras backend
if(K.image_dim_ordering()=="th"): # for Theano
    X_train = train[:, 1:].reshape(train.shape[0], 1, img_rows, img_cols)
    X_test = test[:, 1:].reshape(test.shape[0], 1, img_rows, img_cols)
    in_shape = (1, img_rows, img_cols)
else:  # for TensorFlow
    X_train = train[:, 1:].reshape(train.shape[0], img_rows, img_cols, 1)
    X_test = test[:, 1:].reshape(test.shape[0], img_rows, img_cols, 1)
    in_shape = (img_rows, img_cols, 1)

# First data is label (already removed from X_train)
y_train = train[:, 0] 

# Make the value floats in [0;1] instead of int in [0;255]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices (ie one-hot vectors)
Y_train = np_utils.to_categorical(y_train, nb_classes)

# Display the shapes to check if everything's ok
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_test shape:', X_test.shape)

model = Sequential()

# Add padding to take 28x28 to 32x32
model.add(ZeroPadding2D((2,2),input_shape=in_shape))

# Roughly equivalent to C1
model.add(Convolution2D(6, (5, 5), activation = 'relu',  kernel_initializer='he_normal'))

# Roughly equivalent to S2
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Activation("sigmoid"))

# model.add(Convolution2D(6, (2, 2), stride = (2,2),  activation = 'sigmoid', kernel_initializer='he_normal'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# Roughly equivalent to C3
model.add(Convolution2D(16, (5, 5), activation = 'relu', kernel_initializer='he_normal'))

# Roughly equivalent to S4
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Activation("sigmoid"))

model.add(Dropout(0.2))

# model.add(Convolution2D(16, (2, 2), stride = (2,2),  activation = 'sigmoid', kernel_initializer='he_normal'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# Roughly equivalent to C5
model.add(Convolution2D(120, (5, 5), activation = 'relu', kernel_initializer='he_normal'))
model.add(Flatten())

# Roughly equivalent to F6
model.add(Dense(84, activation = 'tanh', kernel_initializer='he_normal'))


# Output Layer
model.add(Dense(nb_classes, activation = 'softmax', kernel_initializer='he_normal')) #Last layer with one output per class

# Use RMSprop for training weights
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=["accuracy"])

# Alternative training approach using stochastic gradient descent (very very slow)
# sgd = SGD(lr=0.4,momentum=0.1)
# model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])

# Let's Learn!!
model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1)

# Use the test data to see how we do
yPred = model.predict_classes(X_test)


X_train shape: (60000, 28, 28, 1)
Y_train shape: (60000, 10)
X_test shape: (10000, 28, 28, 1)
Epoch 1/4
60000/60000 [==============================] - 44s - loss: 1.8739 - acc: 0.3209      

So it does not appear at first glance that using a larger training set is giving much different results. 

# Model 2:

This version replaces AveragePooling with MaxPooling and removes the dropout layer (which I do not believe was a part of the LeNet specification).

The original LeNet implementation uses average pooling, but let's try max pooling and see if it makes a difference. According to here:
https://www.quora.com/What-is-a-downsampling-layer-in-Convolutional-Neural-Network-CNN

and several other similar sources, max pooling is used more commonly because of better performance. My understand of max pooling is that it better preserves contrast, which we would expect to be important in recognizing image features.

In [6]:
model2 = Sequential()

# Add padding to take 28x28 to 32x32
model2.add(ZeroPadding2D((2,2),input_shape=in_shape))

model2.add(Convolution2D(6, (5, 5), activation = 'relu',  kernel_initializer='he_normal'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Activation("sigmoid"))

model2.add(Convolution2D(16, (5, 5), activation = 'relu', kernel_initializer='he_normal'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Activation("sigmoid"))

#model2.add(Dropout(0.2))

model2.add(Convolution2D(120, (5, 5), activation = 'relu', kernel_initializer='he_normal'))
model2.add(Flatten())

model2.add(Dense(84, activation = 'tanh', kernel_initializer='he_normal'))
model2.add(Dense(nb_classes, activation = 'softmax', kernel_initializer='he_normal')) #Last layer with one output per class

# Use RMSprop for training weights
model2.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=["accuracy"])
model2.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1)

Epoch 1/4
60000/60000 [==============================] - 39s - loss: 1.7966 - acc: 0.3593      

Model 2 gives a noticable improvement in performance, but the results are still not very good.

# Model 3:

In [8]:
model3 = Sequential()

# first set of convolutional => rectifier => pooling
model3.add(Convolution2D(20, 5, 5, border_mode="same",input_shape=in_shape))
model3.add(Activation("relu"))
model3.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of convolutional => rectifier => pooling
model3.add(Convolution2D(50, 5, 5, border_mode="same"))
model3.add(Activation("relu"))
model3.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# set of full => rectifier layers
model3.add(Flatten())
model3.add(Dense(500))
model3.add(Activation("relu"))

# softmax classifier
model3.add(Dense(10))
model3.add(Activation("softmax"))

model3.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=["accuracy"])
model3.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1)

/Users/gdcutting/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), input_shape=(28, 28, 1..., padding="same")`
/Users/gdcutting/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`


Epoch 1/4
60000/60000 [==============================] - 143s - loss: 0.1431 - acc: 0.9550     

Here we get rid of the initial padding layer. This model uses rectifier activation instead of softmax. It also adds some neurons to the various layers (this version is overspecified compare to LeNet-5).

This model performs MUCH better than the previous ones(>99% accuracy).

Let's look at the confusion matrix for this model:

In [9]:
targets = test[:,0]

cm = np.array([[0] * 10] * 10)
for i in range(len(targets)):
 cm[yPred[i],targets[i]] += 1

print(cm)

[[ 948    0    9    7    1   12    7    1   10    5]
 [   0 1115    6    9    3   20    3   27   24    6]
 [   0    3  907   22    1    1    2   34    5    1]
 [   0    2   13  892    0   34    1    2   11    4]
 [   5    2   37    4  921   55   10   22   28  235]
 [   1    0    0   16    0  689    4    0   11    4]
 [   8    6   17    3   17   21  930    0   15    2]
 [   1    0   18   11    0    4    0  832    8    3]
 [  13    6   22   27    0   22    1    3  818    8]
 [   4    1    3   19   39   34    0  107   44  741]]


The confusion matrix demostrates the increased accuracy of this model over the first one. 

From the fifth row, we can see that a lot of 4s are misclassified as 9s (which is not surprising). Also, from the last row, we see that a lot of 9s are misclassified as 7s.

# Model 4:

In [6]:
model4 = Sequential()

# first set of convolutional => rectifier => pooling
model4.add(Convolution2D(6, 5, 5, border_mode="same",input_shape=in_shape))
model4.add(Activation("relu"))
model4.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of convolutional => rectifier => pooling
model4.add(Convolution2D(16, 5, 5, border_mode="same"))
model4.add(Activation("relu"))
model4.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# set of full => rectifier layers
model4.add(Flatten())
model4.add(Dense(120))
model4.add(Activation("relu"))
model4.add(Dense(84))
model4.add(Activation("relu"))

# softmax classifier
model4.add(Dense(10))
model4.add(Activation("softmax"))

model4.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=["accuracy"])
model4.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1)

/Users/gdcutting/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), input_shape=(28, 28, 1..., padding="same")`
/Users/gdcutting/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), padding="same")`


Epoch 1/4
60000/60000 [==============================] - 43s - loss: 0.2735 - acc: 0.9149      

This version is closest to the exact LeNet-5 architecture specified in the Lecun paper. Specifically, it retains the exact numbers of neurons in each layer (6, 16, 120, 84, 10). It uses MaxPooling instead of AveragePooling. Performance is slightly reduced as compared to Model 3 (98.9% vs. 99.4%). Does not include zero padding though.